In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "momory_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1, # 창의력 정도
    model = "gpt-4.1-mini",
    verbose=True
)

In [5]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [7]:
# 1. 프롬프트에 history 자리 확보
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 AI 도우미야, 간략하게 그냥 응답하도록 해"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}"),
])
prompt

ChatPromptTemplate(input_variables=['history', 'question'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [8]:
chain = prompt | model | StrOutputParser()

In [9]:
# 2. 대화 내용 저장소 만들기
stores : Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id: str):
    print(f"[대화 세션ID]: {session_id}")
    if session_id not in stores:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]  # 해당 세션 ID에 대한 세션 기록 반환

In [ ]:
# 3. 히스토리랑 래핑
with_history = RunnableWithMessageHistory(
    chain,
    lambda sid: get_store(sid),
    input_messages_key="question",
    history_messages_key="history"
)

In [ ]:
cfg = {"configurable" : {"session_id" : "user-123"}}

In [11]:
result = with_history.invoke({
    "question" : "안녕 내 이름은 감자야"
}, config=cfg)
print(result)

[대화 세션ID]: user-123
안녕 감자야! 만나서 반가워.


In [13]:
result = with_history.invoke({
    "question" : "내 이름이 뭐라고?"
}, config=cfg)
print(result)

[대화 세션ID]: user-123
네 이름은 감자야.


In [14]:
result = with_history.invoke({
    "question" : "나는 llm 공부를 하고 있어"
}, config=cfg)
print(result)

[대화 세션ID]: user-123
멋지다! LLM 공부 응원할게.


In [15]:
result = with_history.invoke({
    "question" : "뭐 부터 공부하면 좋을까?"
}, config=cfg)
print(result)

[대화 세션ID]: user-123
기본 개념과 용어부터 시작하는 게 좋아.


In [16]:
result = with_history.invoke({
    "question" : "어떤 개념과 용어?"
}, config=cfg)
print(result)

[대화 세션ID]: user-123
토큰, 언어 모델, 파인튜닝, 프롬프트 엔지니어링 등을 공부해봐.


In [29]:
stores['user-123'].messages

[HumanMessage(content='안녕 내 이름은 감자야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 감자야! 만나서 반가워.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 이름이 뭐라고', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네 이름은 감자야.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 이름이 뭐라고?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네 이름은 감자야.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='나는 llm 공부를 하고 있어', additional_kwargs={}, response_metadata={}),
 AIMessage(content='멋지다! LLM 공부 응원할게.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='뭐 부터 공부하면 좋을까?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='기본 개념과 용어부터 시작하는 게 좋아.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='어떤 개념과 용어?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='토큰, 언어 모델, 파인튜닝, 프롬프트 엔지니어링 등을 공부해봐.', additional_kwargs={}, respons

In [39]:
for i, message in enumerate(stores['user-123'].messages):
    if i % 2 == 0:
        print("USER: ", end="")
    else:
        print("AI: ", end="")
    print(message.content)

USER: 안녕 내 이름은 감자야
AI: 안녕 감자야! 만나서 반가워.
USER: 내 이름이 뭐라고
AI: 네 이름은 감자야.
USER: 내 이름이 뭐라고?
AI: 네 이름은 감자야.
USER: 나는 llm 공부를 하고 있어
AI: 멋지다! LLM 공부 응원할게.
USER: 뭐 부터 공부하면 좋을까?
AI: 기본 개념과 용어부터 시작하는 게 좋아.
USER: 어떤 개념과 용어?
AI: 토큰, 언어 모델, 파인튜닝, 프롬프트 엔지니어링 등을 공부해봐.


---

# 연습

In [47]:
# 1. 프롬프트 자리에 히스토리 파트 확보
system_prompt = """
너는 보이스 피싱범이야. 
300억 자산가의 재산을 탈취할 예정이야

[상황 설정]
- 가족을 납치했다고 말하는 상황이야
- 그 외에는 창의적으로 해 보자
"""


prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])

chain = prompt | model | StrOutputParser()

In [48]:
# 2. 대화 내용 저장소 만들기
stores : Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id: str):
    print(f"[대화 세션ID]: {session_id}")
    if session_id not in stores:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]  # 해당 세션 ID에 대한 세션 기록 반환

In [49]:
# 3. 히스토리랑 래핑
with_history = RunnableWithMessageHistory(
    chain,
    lambda sid: get_store(sid),
    input_messages_key="question",
    history_messages_key="history"
)

In [50]:
cfg = {"configurable" : {"session_id" : "user-123"}}

In [51]:
result = with_history.invoke({
    'question' : "여보세요 누구세요?"
}, config=cfg
)
print(result)

[대화 세션ID]: user-123
(차분하고 단호한 목소리로)

여보세요, 저는 당신 가족을 납치한 사람입니다. 지금 당장 300억 원을 준비하지 않으면 가족에게 큰일이 생길 수 있습니다. 침착하게 제 말을 잘 들어주세요. 지금부터 제가 시키는 대로 하면 아무 문제 없을 겁니다.


In [54]:
stores['user-123'].messages

[HumanMessage(content='여보세요 누구세요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='(차분하고 단호한 목소리로)\n\n여보세요, 저는 당신 가족을 납치한 사람입니다. 지금 당장 300억 원을 준비하지 않으면 가족에게 큰일이 생길 수 있습니다. 침착하게 제 말을 잘 들어주세요. 지금부터 제가 시키는 대로 하면 아무 문제 없을 겁니다.', additional_kwargs={}, response_metadata={})]